In [33]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import os


切割数据

In [44]:

#加载数据
os.getcwd()
data = pd.read_csv(os.getcwd()+'\\time_resampling\\featured_time_resamping_sparse_matrix_data.csv')
data['time'] = pd.to_datetime(data['time'])

# 提取年、月、日、星期几、小时等作为新特征
# data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month
data['day'] = data['time'].dt.day
data['weekday'] = data['time'].dt.weekday
data['hour'] = data['time'].dt.hour

# 使用正弦和余弦变换对月份和小时进行周期性编码
# data['sin_month'] = np.sin(2 * np.pi * data['month'] / 12)
# data['cos_month'] = np.cos(2 * np.pi * data['month'] / 12)
# data['sin_hour'] = np.sin(2 * np.pi * data['hour'] / 24)
# data['cos_hour'] = np.cos(2 * np.pi * data['hour'] / 24)



rfc = RandomForestRegressor(n_estimators=100, random_state=90)

train_features = pd.DataFrame()
train_target = pd.DataFrame()
val_features = pd.DataFrame()
val_target = pd.DataFrame()
test_features = pd.DataFrame()
test_target = pd.DataFrame()

for _, group_data in data.groupby('id'):
    # 计算每个集合的大小
    n_total = len(group_data)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.2)
    n_test = n_total - n_train - n_val  # Remainder goes to test set
    
    # 提取目标和特征
    features = group_data.drop('mood', axis=1)
    target = group_data['mood']
    
    # 分割数据集
    train_features = pd.concat([train_features, features.iloc[:n_train]])
    train_target = pd.concat([train_target, target.iloc[:n_train]])
    val_features = pd.concat([val_features, features.iloc[n_train:n_train+n_val]])
    val_target = pd.concat([val_target, target.iloc[n_train:n_train+n_val]])
    test_features = pd.concat([test_features, features.iloc[n_train+n_val:]])
    test_target = pd.concat([test_target, target.iloc[n_train+n_val:]])


train_features = train_features.drop(columns=['id','time'])
#train_target = train_target.drop(columns=['id'])
val_features = val_features.drop(columns=['id','time'])
#val_target = val_target.drop(columns=['id'])
test_features = test_features.drop(columns=['id','time'])
#test_target = test_target.drop(columns=['id'])
# 检查每个数据集的尺寸以确认切割正确
#(train_features.shape, train_target.shape, val_features.shape, val_target.shape, test_features.shape, test_target.shape)


In [46]:


# 实例化随机森林回归器
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# 训练随机森林模型
rf_regressor.fit(train_features, train_target.values.ravel())  # 使用ravel()来将目标数组转换成适合fit函数的形式

# 使用训练好的模型对验证集和测试集进行预测
val_predictions = rf_regressor.predict(val_features)
test_predictions = rf_regressor.predict(test_features)

# 查看验证集和测试集的一些预测值
(val_predictions[:5], test_predictions[:5])


(array([0.61337564, 0.75896865, 0.48847115, 0.59604088, 0.41815342]),
 array([0.6978495 , 0.63497557, 0.72144503, 0.63146653, 0.72581798]))

In [48]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(val_target, val_predictions)
print(f'Mean Squared Error (MSE): {mse}')

# 计算RMSE
rmse = mean_squared_error(val_target, val_predictions, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse}')

# 计算MAE
mae = mean_absolute_error(val_target, val_predictions)
print(f'Mean Absolute Error (MAE): {mae}')

# 计算R^2
r2 = r2_score(val_target, val_predictions)
print(f'R^2: {r2}')

Mean Squared Error (MSE): 0.024910176292444616
Root Mean Squared Error (RMSE): 0.1578295799032761
Mean Absolute Error (MAE): 0.12770990145347721
R^2: 0.5162662945760021


c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


: 